# Fisher test

In [40]:
import os, sys, time
import numpy as np
import scipy
import healpy as hp
from fgbuster.observation_helpers import *

import micmac

In [5]:
working_directory_path = os.path.abspath('') + '/'
# directory_save_file = working_directory_path + 'save_directory/'
directory_toml_file = working_directory_path #+ 'toml_params/'
path_toml_file = directory_toml_file + 'full_chain_v1a.toml'

In [6]:
MICMAC_sampler_obj = micmac.create_MICMAC_sampler_from_toml_file(path_toml_file)

In [7]:
# General parameters
cmb_model = 'c1'
fgs_model = 'd0s0'
model = cmb_model+fgs_model
noise = True
# noise = False
noise_seed = 42
# instr_name = dictionary_parameters['instrument_name'] #'SO_SAT'
instr_name = 'SO_SAT'

# get instrument from public database
instrument = get_instrument(instr_name)

# get input freq maps
np.random.seed(noise_seed)
# freq_maps = get_observation(instrument, model, nside=NSIDE, noise=noise)[:, 1:, :]   # keep only Q and U
freq_maps_fgs = get_observation(instrument, fgs_model, nside=MICMAC_sampler_obj.nside, noise=noise)[:, 1:, :]   # keep only Q and U
print("Shape for input frequency maps :", freq_maps_fgs.shape)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Shape for input frequency maps : (6, 2, 49152)


In [8]:
init_mixing_matrix_obj = micmac.InitMixingMatrix(MICMAC_sampler_obj.frequency_array, MICMAC_sampler_obj.number_components, pos_special_freqs=MICMAC_sampler_obj.pos_special_freqs)

exact_params_mixing_matrix = init_mixing_matrix_obj.init_params()

mixing_matrix_obj = micmac.MixingMatrix(MICMAC_sampler_obj.frequency_array, MICMAC_sampler_obj.number_components, exact_params_mixing_matrix, pos_special_freqs=MICMAC_sampler_obj.pos_special_freqs)
# invBtinvNB = micmac.get_inv_BtinvNB(MICMAC_sampler_obj.freq_inverse_noise, mixing_matrix_obj.get_B())

In [9]:
input_freq_maps, input_cmb_maps, theoretical_red_cov_r0_total, theoretical_red_cov_r1_tensor = MICMAC_sampler_obj.generate_input_freq_maps_from_fgs(freq_maps_fgs, return_only_freq_maps=False)

Calculating spectra from CAMB !
Calculating spectra from CAMB !


In [10]:

theoretical_r0_total = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r0_total)#[partial_indices_polar,:]
theoretical_r1_tensor = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r1_tensor)#[partial_indices_polar,:]

indices_polar = np.array([1,2,4])
partial_indices_polar = indices_polar[:MICMAC_sampler_obj.nstokes]

red_covar_CMB = theoretical_red_cov_r0_total + MICMAC_sampler_obj.r_true * theoretical_red_cov_r1_tensor

In [11]:
exact_params_mixing_matrix, exact_params_mixing_matrix.ravel(order='F')

(array([[0.33858815, 0.00979233],
        [0.02963068, 0.05147511],
        [0.00998213, 0.12913911],
        [0.00328725, 0.44076761]]),
 array([0.33858815, 0.02963068, 0.00998213, 0.00328725, 0.00979233,
        0.05147511, 0.12913911, 0.44076761]))

In [12]:
input_freq_maps_extended = np.insert(input_freq_maps, [0], np.zeros(MICMAC_sampler_obj.npix), axis=1)


# Setting new noise

In [25]:
B_matrix = mixing_matrix_obj.get_B()

In [46]:
freq_noise = np.linalg.pinv(MICMAC_sampler_obj.freq_inverse_noise)
red_freq_noise = np.einsum('fk,l,sp->fklsp', freq_noise, np.ones_like(red_covar_CMB[:,0,0]), np.eye(MICMAC_sampler_obj.nstokes))

def get_logproba_spectral(param_mixing_matrix, red_cell_CMB, red_cell_noise, _input_freq_maps):

    mixing_matrix_obj.update_params(param_mixing_matrix.reshape((MICMAC_sampler_obj.number_frequencies-len(MICMAC_sampler_obj.pos_special_freqs), MICMAC_sampler_obj.number_components-1), order='F'))
    mixing_matrix = mixing_matrix_obj.get_B()

    B_f = mixing_matrix[:,1:]
    
    red_cmb_noise = np.einsum('k,lsp,f->kflsp', B_matrix[:,0], red_cell_CMB, B_matrix[:,0])
    total_noise = red_cmb_noise + red_cell_noise

    inverse_central_term = np.linalg.pinv(np.einsum('fc,fk...,kp->cp...', B_f, np.linalg.pinv(total_noise), B_f))

    global_central_term = np.einsum('fklst,kc,cmltu,im,ijluv->fjlsv', np.linalg.pinv(total_noise), B_f, inverse_central_term, B_f, np.linalg.pinv(total_noise))

    modified_map = np.zeros_like(_input_freq_maps)
    for freq1 in range(MICMAC_sampler_obj.number_frequencies):
        for freq2 in range(MICMAC_sampler_obj.number_frequencies):
            modified_map[freq1] += micmac.maps_x_red_covariance_cell_JAX(_input_freq_maps[freq2], global_central_term[freq1,freq2], nside=MICMAC_sampler_obj.nside, lmin=MICMAC_sampler_obj.lmin, n_iter=MICMAC_sampler_obj.n_iter)
    return np.einsum('fsp,fsp', _input_freq_maps, modified_map)/2


In [48]:
get_logproba_spectral(exact_params_mixing_matrix, red_covar_CMB, red_freq_noise, input_freq_maps)

70671116831678.11

In [49]:
results_minimizer = scipy.optimize.minimize(get_logproba_spectral, exact_params_mixing_matrix.ravel(order='F'), args=(red_covar_CMB, red_freq_noise, input_freq_maps)) #, method='Nelder-Mead', options={'maxiter': 1000, 'disp': True}

In [50]:
print(results_minimizer.x)

[0.62002271 0.60349808 0.7600056  1.41400384 0.02746361 0.01029355
 0.06526709 0.29676381]


In [51]:
exact_params_mixing_matrix.ravel(order='F')

array([0.33858815, 0.02963068, 0.00998213, 0.00328725, 0.00979233,
       0.05147511, 0.12913911, 0.44076761])